In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from env import get_db_url
import wrangle_zillow as w
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
# creating df variable

In [3]:
df = w.get_zillow_data()

In [4]:
columns = ['buildingqualitytypeid','heatingorsystemdesc','heatingorsystemtypeid','unitcnt','propertyzoningdesc','propertylandusedesc','parcelid.1','id.1','censustractandblock']

In [5]:
#using handle missing values function
df = w.data_prep(df,columns)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50604 entries, 0 to 52318
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   propertylandusetypeid       50604 non-null  float64
 1   parcelid                    50604 non-null  int64  
 2   id                          50604 non-null  int64  
 3   baths                       50604 non-null  float64
 4   beds                        50604 non-null  float64
 5   calculatedbathnbr           50604 non-null  float64
 6   sqft                        50604 non-null  float64
 7   finishedsquarefeet12        50604 non-null  float64
 8   fips                        50604 non-null  float64
 9   fullbathcnt                 50604 non-null  float64
 10  latitude                    50604 non-null  float64
 11  longitude                   50604 non-null  float64
 12  lotsizesquarefeet           50604 non-null  float64
 13  propertycountylandusecode   506

In [7]:
df = w.prepare_zillow(df)

/Users/zacharystufflebeme/codeup-data-science/zach-allante/clustering-project/wrangle_zillow.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.orange_county[df.location == 1] = 1
/Users/zacharystufflebeme/codeup-data-science/zach-allante/clustering-project/wrangle_zillow.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ventura[df.location == 2] = 1
/Users/zacharystufflebeme/codeup-data-science/zach-allante/clustering-project/wrangle_zillow.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50604 entries, 0 to 52318
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   propertylandusetypeid       50604 non-null  float64
 1   parcelid                    50604 non-null  int64  
 2   id                          50604 non-null  int64  
 3   baths                       50604 non-null  float64
 4   beds                        50604 non-null  float64
 5   calculatedbathnbr           50604 non-null  float64
 6   sqft                        50604 non-null  float64
 7   finishedsquarefeet12        50604 non-null  float64
 8   fips                        50604 non-null  object 
 9   fullbathcnt                 50604 non-null  float64
 10  latitude                    50604 non-null  float64
 11  longitude                   50604 non-null  float64
 12  lotsizesquarefeet           50604 non-null  float64
 13  propertycountylandusecode   506

In [9]:
train, validate, test = w.wrangle_zillow(df)

In [10]:
train

,propertylandusetypeid,parcelid,id,baths,beds,calculatedbathnbr,sqft,finishedsquarefeet12,fips,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,max_transactiondate,logerror,transactiondate,location,orange_county,ventura,losangeles
13535,261.0,12397906,66906,1.0,2.0,1.0,872.0,872.0,Los Angeles CA,1.0,...,2016.0,266297.0,4357.93,2017-03-24,0.026998,2017-03-24,0,0,0,1
30453,261.0,14380619,2800936,3.0,3.0,3.0,2260.0,2260.0,Orange County CA,3.0,...,2016.0,142053.0,3515.14,2017-06-09,0.037181,2017-06-09,1,1,0,0
13337,261.0,12902896,1127949,2.0,3.0,2.0,1091.0,1091.0,Los Angeles CA,2.0,...,2016.0,50642.0,1893.90,2017-03-24,0.019205,2017-03-24,0,0,0,1
7660,261.0,11568321,2658421,6.0,5.0,6.0,4783.0,4783.0,Los Angeles CA,6.0,...,2016.0,2288518.0,47785.18,2017-02-22,0.037875,2017-02-22,0,0,0,1
34175,261.0,13100555,1758804,5.0,5.0,5.0,4425.0,4425.0,Los Angeles CA,5.0,...,2016.0,540465.0,14379.50,2017-06-27,0.002652,2017-06-27,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38066,261.0,14238880,638264,2.0,3.0,2.0,1611.0,1611.0,Orange County CA,2.0,...,2016.0,548239.0,7041.04,2017-07-12,-0.006377,2017-07-12,1,1,0,0
48425,261.0,12438510,2110521,3.0,5.0,3.0,2652.0,2652.0,Los Angeles CA,3.0,...,2016.0,414596.0,9365.87,2017-08-30,-0.008494,2017-08-30,0,0,0,1
19647,261.0,11508139,2077101,2.0,3.0,2.0,1962.0,1962.0,Los Angeles CA,2.0,...,2016.0,276422.0,4630.60,2017-04-25,0.680864,2017-04-25,0,0,0,1
21717,261.0,14314819,161303,2.0,3.0,2.0,1714.0,1714.0,Orange County CA,2.0,...,2016.0,271335.0,4590.46,2017-05-03,0.038707,2017-05-03,1,1,0,0


In [11]:
trainExplore = train
trainExplore['logerror_bin'] = pd.cut(train.logerror, [-5,-1,-.75,-.5,-.25,0,.25,.50,.75, 1, 5])
trainExplore['latitude_bin'] = pd.cut(train.latitude, [0,33600000,34200000,34600000,35000000])
trainExplore['longitude_bin'] = pd.cut(train.longitude, [-119200000,-119000000,-118600000,-117800000,0])
trainExplore['squarefeet_bin'] = pd.cut(train.sqft,[0,3000,8000,12000])
trainExplore['age_bin'] = pd.cut(train.age, [0,30,60,90,120])
trainExplore['lowerrors'] = pd.cut(train.logerror, [-.25,0,.25,.5])
train = train[train.lotsizesquarefeet < 100000]

In [12]:
impfeats = ['sqft','age','latitude','longitude','lotsizesquarefeet']


In [13]:
trainv= train.dropna()

In [14]:
imp_df = train[impfeats]

In [15]:
imp_df.head()

,sqft,age,latitude,longitude,lotsizesquarefeet
13535,872.0,71.0,33957440.0,-118094485.0,5180.0
30453,2260.0,46.0,33616842.0,-117684836.0,7000.0
13337,1091.0,61.0,34101957.0,-117830613.0,7487.0
7660,4783.0,9.0,34044340.0,-118470565.0,10689.0
34175,4425.0,23.0,33991489.0,-117846502.0,10375.0


In [16]:
train

,propertylandusetypeid,parcelid,id,baths,beds,calculatedbathnbr,sqft,finishedsquarefeet12,fips,fullbathcnt,...,location,orange_county,ventura,losangeles,logerror_bin,latitude_bin,longitude_bin,squarefeet_bin,age_bin,lowerrors
13535,261.0,12397906,66906,1.0,2.0,1.0,872.0,872.0,Los Angeles CA,1.0,...,0,0,0,1,"(0.0, 0.25]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(60, 90]","(0.0, 0.25]"
30453,261.0,14380619,2800936,3.0,3.0,3.0,2260.0,2260.0,Orange County CA,3.0,...,1,1,0,0,"(0.0, 0.25]","(33600000, 34200000]","(-117800000, 0]","(0, 3000]","(30, 60]","(0.0, 0.25]"
13337,261.0,12902896,1127949,2.0,3.0,2.0,1091.0,1091.0,Los Angeles CA,2.0,...,0,0,0,1,"(0.0, 0.25]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(60, 90]","(0.0, 0.25]"
7660,261.0,11568321,2658421,6.0,5.0,6.0,4783.0,4783.0,Los Angeles CA,6.0,...,0,0,0,1,"(0.0, 0.25]","(33600000, 34200000]","(-118600000, -117800000]","(3000, 8000]","(0, 30]","(0.0, 0.25]"
34175,261.0,13100555,1758804,5.0,5.0,5.0,4425.0,4425.0,Los Angeles CA,5.0,...,0,0,0,1,"(0.0, 0.25]","(33600000, 34200000]","(-118600000, -117800000]","(3000, 8000]","(0, 30]","(0.0, 0.25]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38066,261.0,14238880,638264,2.0,3.0,2.0,1611.0,1611.0,Orange County CA,2.0,...,1,1,0,0,"(-0.25, 0.0]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(60, 90]","(-0.25, 0.0]"
48425,261.0,12438510,2110521,3.0,5.0,3.0,2652.0,2652.0,Los Angeles CA,3.0,...,0,0,0,1,"(-0.25, 0.0]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(30, 60]","(-0.25, 0.0]"
19647,261.0,11508139,2077101,2.0,3.0,2.0,1962.0,1962.0,Los Angeles CA,2.0,...,0,0,0,1,"(0.5, 0.75]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(60, 90]",NaN
21717,261.0,14314819,161303,2.0,3.0,2.0,1714.0,1714.0,Orange County CA,2.0,...,1,1,0,0,"(0.0, 0.25]","(33600000, 34200000]","(-118600000, -117800000]","(0, 3000]","(30, 60]","(0.0, 0.25]"


In [17]:
scaled_impfeats = w.scale_zillow(imp_df,impfeats )

/Users/zacharystufflebeme/codeup-data-science/zach-allante/clustering-project/wrangle_zillow.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impdf[impfeats] = Scaler.fit_transform(impdf[impfeats])


In [18]:
clusters = pd.DataFrame([])
for i in range(2,10):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(scaled_impfeats[impfeats])
    train['clusters',i] = kmeans.predict(scaled_impfeats[impfeats])
    clusters['clusters',i] = kmeans.predict(scaled_impfeats)

/var/folders/yn/3nqd95h90wd7cdqr1w2mwmz40000gn/T/ipykernel_99763/779291006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clusters',i] = kmeans.predict(scaled_impfeats[impfeats])
/var/folders/yn/3nqd95h90wd7cdqr1w2mwmz40000gn/T/ipykernel_99763/779291006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clusters',i] = kmeans.predict(scaled_impfeats[impfeats])
/var/folders/yn/3nqd95h90wd7cdqr1w2mwmz40000gn/T/ipykernel_99763/779291006.py:5: SettingWithCopyWarning: 
A value is trying to be 

In [19]:
train

,propertylandusetypeid,parcelid,id,baths,beds,calculatedbathnbr,sqft,finishedsquarefeet12,fips,fullbathcnt,...,age_bin,lowerrors,"(clusters, 2)","(clusters, 3)","(clusters, 4)","(clusters, 5)","(clusters, 6)","(clusters, 7)","(clusters, 8)","(clusters, 9)"
13535,261.0,12397906,66906,1.0,2.0,1.0,872.0,872.0,Los Angeles CA,1.0,...,"(60, 90]","(0.0, 0.25]",0,0,1,1,1,6,1,0
30453,261.0,14380619,2800936,3.0,3.0,3.0,2260.0,2260.0,Orange County CA,3.0,...,"(30, 60]","(0.0, 0.25]",0,1,2,0,2,0,3,4
13337,261.0,12902896,1127949,2.0,3.0,2.0,1091.0,1091.0,Los Angeles CA,2.0,...,"(60, 90]","(0.0, 0.25]",0,0,1,1,1,6,1,6
7660,261.0,11568321,2658421,6.0,5.0,6.0,4783.0,4783.0,Los Angeles CA,6.0,...,"(0, 30]","(0.0, 0.25]",1,2,3,2,4,3,7,7
34175,261.0,13100555,1758804,5.0,5.0,5.0,4425.0,4425.0,Los Angeles CA,5.0,...,"(0, 30]","(0.0, 0.25]",0,1,2,0,2,3,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38066,261.0,14238880,638264,2.0,3.0,2.0,1611.0,1611.0,Orange County CA,2.0,...,"(60, 90]","(-0.25, 0.0]",0,1,2,1,1,6,1,0
48425,261.0,12438510,2110521,3.0,5.0,3.0,2652.0,2652.0,Los Angeles CA,3.0,...,"(30, 60]","(-0.25, 0.0]",0,1,2,1,1,6,1,0
19647,261.0,11508139,2077101,2.0,3.0,2.0,1962.0,1962.0,Los Angeles CA,2.0,...,"(60, 90]",NaN,0,0,1,3,1,6,1,0
21717,261.0,14314819,161303,2.0,3.0,2.0,1714.0,1714.0,Orange County CA,2.0,...,"(30, 60]","(0.0, 0.25]",0,1,2,0,2,0,3,4


In [20]:
imp_df.head()

,sqft,age,latitude,longitude,lotsizesquarefeet
13535,0.060570,0.496183,0.433728,0.721678,0.049851
30453,0.177337,0.305344,0.194141,0.940035,0.068203
13337,0.078994,0.419847,0.535385,0.862331,0.073113
7660,0.389585,0.022901,0.494856,0.521215,0.105400
34175,0.359468,0.129771,0.457679,0.853861,0.102233


In [21]:
clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28162 entries, 0 to 28161
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   (clusters, 2)  28162 non-null  int32
 1   (clusters, 3)  28162 non-null  int32
 2   (clusters, 4)  28162 non-null  int32
 3   (clusters, 5)  28162 non-null  int32
 4   (clusters, 6)  28162 non-null  int32
 5   (clusters, 7)  28162 non-null  int32
 6   (clusters, 8)  28162 non-null  int32
 7   (clusters, 9)  28162 non-null  int32
dtypes: int32(8)
memory usage: 880.2 KB
